# DuckDB Explorer Notebook

This notebook provides an interactive way to explore the tables in your DuckDB database.

In [ ]:
import duckdb
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Set up plotting
%matplotlib inline
plt.style.use("ggplot")
sns.set(style="whitegrid")

## Connect to the Database

First, let's connect to the DuckDB database:

In [ ]:
# Update this path to your DuckDB database file
DB_PATH = "../data/financial_data.duckdb"

# Connect to the database
conn = duckdb.connect(DB_PATH)
print(f"Connected to DuckDB database: {DB_PATH}")

## List Tables

Let's list all the tables in the database:

In [ ]:
# List all tables
tables = conn.execute("SHOW TABLES").fetchall()
tables_df = pd.DataFrame(tables, columns=["Table Name"])
tables_df

## Explore Table Schema

Let's explore the schema of each table:

In [ ]:
# Function to get table schema
def get_table_schema(table_name):
    schema = conn.execute(f"PRAGMA table_info('{table_name}')").fetchall()
    schema_df = pd.DataFrame(
        schema, columns=["cid", "name", "type", "notnull", "dflt_value", "pk"]
    )
    schema_df["nullable"] = ~schema_df["notnull"].astype(bool)
    schema_df["primary_key"] = schema_df["pk"].astype(bool)
    return schema_df[["name", "type", "nullable", "dflt_value", "primary_key"]]


# Get schema for each table
for table in tables:
    table_name = table[0]
    print(f"\n## Schema for '{table_name}'")
    schema_df = get_table_schema(table_name)
    display(schema_df)

    # Show row count
    count = conn.execute(f"SELECT COUNT(*) FROM '{table_name}'").fetchone()[0]
    print(f"Row count: {count}")

## Sample Data

Let's look at sample data from each table:

In [ ]:
# Function to get sample data
def get_sample_data(table_name, limit=10):
    sample = conn.execute(f"SELECT * FROM '{table_name}' LIMIT {limit}").fetchall()
    columns = [
        col[1] for col in conn.execute(f"PRAGMA table_info('{table_name}')").fetchall()
    ]
    return pd.DataFrame(sample, columns=columns)


# Get sample data for each table
for table in tables:
    table_name = table[0]
    print(f"\n## Sample data from '{table_name}'")
    sample_df = get_sample_data(table_name)
    display(sample_df)

## Custom Queries

You can run custom SQL queries against the database:

In [ ]:
# Example: Get the number of filings per company
query = """
SELECT 
    c.ticker, 
    c.name, 
    COUNT(f.filing_id) AS filing_count
FROM 
    companies c
LEFT JOIN 
    filings f ON c.ticker = f.ticker
GROUP BY 
    c.ticker, c.name
ORDER BY 
    filing_count DESC
"""

result = conn.execute(query).fetchdf()
result

## Financial Data Analysis

Let's analyze some financial data:

In [ ]:
# Example: Get revenue and net income for a company over time
query = """
SELECT 
    ticker,
    metric_name,
    end_date,
    value
FROM 
    time_series_metrics
WHERE 
    ticker = 'MSFT' AND
    metric_name IN ('Revenue', 'NetIncome') AND
    period_type = 'yearly'
ORDER BY 
    metric_name, end_date
"""

result = conn.execute(query).fetchdf()
result

## Visualize Financial Data

Let's visualize the financial data:

In [ ]:
# Pivot the data for plotting
if not result.empty:
    pivot_df = result.pivot(index="end_date", columns="metric_name", values="value")

    # Plot the data
    fig, ax = plt.subplots(figsize=(12, 6))
    pivot_df.plot(kind="bar", ax=ax)

    plt.title("Microsoft Financial Performance")
    plt.xlabel("Year")
    plt.ylabel("Value (USD)")
    plt.xticks(rotation=45)
    plt.legend(title="Metric")
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization")

## Compare Companies

Let's compare financial metrics across companies:

In [ ]:
# Example: Compare revenue for multiple companies
query = """
SELECT 
    ticker,
    end_date,
    value
FROM 
    time_series_metrics
WHERE 
    metric_name = 'Revenue' AND
    period_type = 'yearly' AND
    ticker IN ('MSFT', 'AAPL', 'GOOGL')
ORDER BY 
    ticker, end_date
"""

result = conn.execute(query).fetchdf()
result

In [ ]:
# Pivot and plot the comparison
if not result.empty:
    pivot_df = result.pivot(index="end_date", columns="ticker", values="value")

    # Plot the data
    fig, ax = plt.subplots(figsize=(12, 6))
    pivot_df.plot(kind="bar", ax=ax)

    plt.title("Revenue Comparison")
    plt.xlabel("Year")
    plt.ylabel("Revenue (USD)")
    plt.xticks(rotation=45)
    plt.legend(title="Company")
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization")

## Close Connection

Finally, let's close the database connection:

In [ ]:
# Close the connection
conn.close()
print("Database connection closed")